In [1]:
!pip install cplex
!pip install docplex


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 16.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.5/633.5 kB 34.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docplex: filename=docplex-2.25.236-py3-none-any.whl size=671350 sha256=bf6f294143cf5e91dcbd276b81f928fac52d2f7cae76279ede2ad261751f71c3
  Stored in directory: /root/.cache/pip/wheels/3b/e5/00/0bf0173d67188fe73a13e3a61412b3f975f60205e3fab93a69
Successfully built docplex


In [ ]:
import pandas as pd
from docplex.cp.model import CpoModel
import cplex
import numpy as np
import math

In [ ]:
# compute the multiplication tensor of A: m by n and B: n by p
def general_multiplication_tensor(N, M, P):
   """Multiplication tensor.
   The multiplication tensor T in {0,1} of size NM x MP x PN
   for the multiplication of two matrices of size NxM and MxP
   """
   T = np.zeros((N * M, M * P, N * P), dtype=np.int64)

   for n in range(N):
       for m in range(M):
           for p in range(P):
               # Convert multi-dimensional indices to flat indices.
               a_index = np.ravel_multi_index((n, m), (N, M))
               b_index = np.ravel_multi_index((m, p), (M, P))
               c_index = np.ravel_multi_index((n, p), (N, P))
               T[a_index, b_index, c_index] = 1

   return T

In [ ]:
# compute the multiplication tensor of A: m by n and B: n by p
T3 = [[[1,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0]], #c1

      [[0,0,0,0,0,0,0,0,0],
       [1,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,1,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0]], #c2

      [[0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [1,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,1,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,1,0,0,0,0,0,0]], #c3

      [[0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0]], #c4

      [[0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,1,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0]], #c5

      [[0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,1,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,1,0,0,0]], #c6

      [[0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0]], #c7

      [[0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0]], #c8

      [[0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,1]] #c9
     ]

In [ ]:
# Get multiplication tensor from the factor matrices U, V, and W
def expand_pd(U, V, W):
    """Expand a polyadic decomposition.
    The polyadic expansion T of the factor matrices U, V, and W is defined by:
        T[i, j, k] = \sum_r U[i, r] * V[j, r] * W[k, r].
    """
    I, J, K, R = U.shape[0], V.shape[0], W.shape[0], U.shape[1]
    T = np.zeros((I, J, K))
    for i in range(I):
        for j in range(J):
            for k in range(K):
                for r in range(R):
                    T[k, i, j] += U[i, r] * V[j, r] * W[k, r]
    return T


In [ ]:
def CP_general(N,M,P,T,R):
    mdl = CpoModel()
    u = [[mdl.integer_var(-1, 1, name="U" + str(i) + "_" + str(r)) for r in range(R)] for i in range(N*M)]
    v = [[mdl.integer_var(-1, 1, name="V" + str(j) + "_" + str(r)) for r in range(R)] for j in range(M*P)]
    w = [[mdl.integer_var(-1, 1, name="W" + str(k) + "_" + str(r)) for r in range(R)] for k in range(N*P)]

    # Matrix Multiplication as tensor operation
    for i in range(N*M):
        for j in range(M*P):
            for k in range(N*P):
                mdl.add(mdl.sum(u[i][r]*v[j][r]*w[k][r] for r in range(R)) == T[k][i][j])
    for r in range(R):
      mdl.add(w[3][r] == 0)
      mdl.add(w[6][r] == 0)
      mdl.add(w[7][r] == 0)
      mdl.add(u[3][r] == 0)
      mdl.add(u[6][r] == 0)
      mdl.add(u[7][r] == 0)
      mdl.add(v[3][r] == 0)
      mdl.add(v[6][r] == 0)
      mdl.add(v[7][r] == 0)

    mdl.set_parameters({'LogPeriod': 100000})
    # mdl.set_parameters({'LogVerbosity': 'Quiet'})

    msol = mdl.solve()
    if msol:
      return [msol,u,v,w]
    else:
      print("Infeasible")
      return -1

In [ ]:
%%time
# Run CP model for 2 by 2 matrices
N = 3
M = 3
P = 3
# T = general_multiplication_tensor(N,M,P)
T = T3
R = 10
solution = CP_general(N,M,P,T,R)

CPU times: user 257 ms, sys: 19.3 ms, total: 277 ms
Wall time: 6.32 s


In [ ]:
%%time
# Run CP model for 2 by 2 matrices
N = 3
M = 3
P = 3
# T = general_multiplication_tensor(N,M,P)
T = T3
R = 9
solution = CP_general(N,M,P,T,R)

 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Satisfiability problem - 243 variables, 810 constraints
 ! LogPeriod            = 100000
 ! Initial process time : 0.04s (0.04s extraction + 0.00s propagation)
 !  . Log search space  : 256.8 (before), 256.8 (after)
 !  . Memory usage      : 993.5 kB (before), 993.5 kB (after)
 ! Using parallel search with 2 workers.
 ! ----------------------------------------------------------------------------
 !               Branches  Non-fixed    W       Branch decision
                     100k          7    1        -1  = W1_5
                     100k         27    2   F     1  = V0_5
                     200k         20    2         0  = V2_1
                     200k         21    1   F     0  = W8_3
                     300k         18    2        -1 != U2_3
                     300k         18    1         0 != U5_3
                     400k         67    2        -1  = U4_3
                     400k         

KeyboardInterrupt: ignored

In [ ]:
# Print solution
if type(solution)!=int:
    sol,U,V,W = solution
    U_sol = np.zeros((M*N,R))
    V_sol = np.zeros((P*M,R))
    W_sol = np.zeros((P*N,R))

    for i in range(M*N):
        for r in range(R):
            U_sol[i,r] = sol[U[i][r]]
    for i in range(P*M):
        for r in range(R):
            V_sol[i,r] = sol[V[i][r]]
    for i in range(P*N):
        for r in range(R):
            W_sol[i,r] = sol[W[i][r]]

    print(U_sol)
    print(V_sol)
    print(W_sol)

    print("\n\n")
    t_constraint_programming = expand_pd(U_sol, V_sol, W_sol)

    print(T)
    print("\n")
    print(t_constraint_programming)

    print("\n\n")
    # check if solution is correct
    if (t_constraint_programming==T).all():
      print("CP got the correct T")

[[ 0.  0.  0. -1.  0.  0.  0. -1.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0. -1.  0.]
 [ 0.  0.  0. -1.  0.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  1.  1. -1.  1.  0. -1. -1.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0. -1.  0. -1.  0.  0.  0.  0.]]
[[ 0. -1.  0.  0.  0.  0. -1. -1.  1.  0.]
 [ 0.  1. -1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1. -1.  0.  1.  0.  0.  0.  0.  1.]
 [ 0.  0.  0. -1.  0. -1.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.  0. -1.  0.  1.  0.]
 [ 0. -1. -1.  0.  0.  0. -1.  0.  0. -1.]
 [ 0.  1.  0.  1. -1. -1.  1.  1. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.